## Train on all four modules
## Predict on 1 module at a time A,B,C,D

In [ ]:
# we know what the feature signature for each module is.
# grab feature signature for A
# extract rows from X_test, y_test that have same signature as A

In [16]:
def featureReader(featuresFile):
    #reads featurefiles and projects 3 columns (ppn, sizeGB, Ytime)
    #reads profiledFile (the table from profiling) and reads all 55 columns
    #generates one dataframe with 58 columns and n rows (=size of profiled data from csv)
    import pandas as pd
    from IPython.display import display

    header='ft1 ft2 ft3 ft4 ft5 ft6 ft7 ft8 ft9 ft10 ft11 ft12 ft13 ft14 ft15 ft16 ft17 \
    ft18 ft24 ft25 ft19 ft39 ft20 ft33 ft21 ft35 ft22 ft23 ft34 ft36 ft37 ft38 ft40 ft41 \
    ft42 ft43 ft44 ft45 ft46 ft48 ft47 ft49 ft51 ft50 ft52 ft53 ft54 ft55 ft26 ft27 ft28 ft29 ft30 ft31 ft32'
    
    def value(item):
        return item[item.find('=')+1:]
    
    print("Reading features from %s" % featuresFile)
    df_features = pd.read_table(featuresFile, header=None, delimiter=',',
                       converters={i:value for i in range(55)},
                       names=header.split())
    df_features = df_features.astype(float)
    return df_features

Reading features from ./features/cdhitdup_singlecmd_features.txt


,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,...,ft53,ft54,ft55,ft26,ft27,ft28,ft29,ft30,ft31,ft32
0,2984.0,1149.0,1350.0,1.0,1794.0,748.0,140.0,865.0,833.0,244.0,...,74.0,2087.0,3.0,83.41,197.65,669.0,230.0,1783.0,26.0,873.0


In [103]:
import pickle
with open('datasetIPPD.pickle', 'rb') as handle:
    data = pickle.load(handle)

#extract X_test, Y_test
X_test = data['X_test']
y_test = data['y_test']

In [104]:
# Signature for A
from IPython.display import display
signatureA = featureReader('./features/cdhitdup_singlecmd_features.txt')
display(signatureA)

Reading features from ./features/cdhitdup_singlecmd_features.txt


,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,...,ft53,ft54,ft55,ft26,ft27,ft28,ft29,ft30,ft31,ft32
0,2984.0,1149.0,1350.0,1.0,1794.0,748.0,140.0,865.0,833.0,244.0,...,74.0,2087.0,3.0,83.41,197.65,669.0,230.0,1783.0,26.0,873.0


In [105]:
# Signature for B
signatureB = featureReader('./features/frhit_singlecmd_features.txt')
display(signatureB)

Reading features from ./features/frhit_singlecmd_features.txt


,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,...,ft53,ft54,ft55,ft26,ft27,ft28,ft29,ft30,ft31,ft32
0,2286.0,985.0,973.0,24.0,1436.0,577.0,113.0,778.0,543.0,166.0,...,104.0,2007.0,5.0,126.11,152.59,223.0,458.0,1445.0,0.0,681.0


## A=cdhitdup, B=frhit, C=velvetG, D=velvetH

In [106]:
# Signature for C
signatureC = featureReader('./features/velvetg_singlecmd_features.txt')
display(signatureC)

Reading features from ./features/velvetg_singlecmd_features.txt


,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,...,ft53,ft54,ft55,ft26,ft27,ft28,ft29,ft30,ft31,ft32
0,9837.0,4069.0,3972.0,78.0,6043.0,2139.0,633.0,3149.0,2417.0,637.0,...,429.0,6062.0,298.0,349.07,777.61,1330.0,1353.0,6132.0,67.0,2616.0


In [107]:
# Signature for D
signatureD = featureReader('./features/velveth_singlecmd_features.txt')
display(signatureD)

Reading features from ./features/velveth_singlecmd_features.txt


,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,...,ft53,ft54,ft55,ft26,ft27,ft28,ft29,ft30,ft31,ft32
0,2301.0,833.0,979.0,14.0,1428.0,503.0,132.0,631.0,628.0,160.0,...,89.0,1295.0,22.0,88.28,195.57,224.0,347.0,1492.0,13.0,558.0


## Extractor for extracting features

In [145]:
cols = ['ft1', 'ft2', 'ft3', 'ft4', 'ft5', 'ft6', 'ft7', 'ft8', 'ft9', 'ft10',
       'ft11', 'ft12', 'ft13', 'ft14', 'ft15', 'ft16', 'ft17', 'ft18', 'ft24',
       'ft25', 'ft19', 'ft39', 'ft20', 'ft33', 'ft21', 'ft35', 'ft22', 'ft23',
       'ft34', 'ft36', 'ft37', 'ft38', 'ft40', 'ft41', 'ft42', 'ft43', 'ft44',
       'ft45', 'ft46', 'ft48', 'ft47', 'ft49', 'ft51', 'ft50', 'ft52', 'ft53',
       'ft54', 'ft55', 'ft26', 'ft27', 'ft28', 'ft29', 'ft30', 'ft31', 'ft32']

In [302]:
len(X_test)

65

In [273]:
def extractor(signature, X_test, y_test):
    import pandas as pd
    import numpy  as np
    import copy
    
    count = 0
    xt = pd.DataFrame(columns = X_test.columns)
    yt = []
    
    for i in range(len(X_test)):
        if(all(X_test[cols].iloc[[i]].values[0] == signature.iloc[[0]].values[0])):
            #print(X_test[cols].iloc[[i]].values[0])
            #print(signature.iloc[[0]].values[0])
            count = count + 1
            # make deep copies
            xt = xt.append(copy.deepcopy(X_test.iloc[[i]]), ignore_index=True)
            yt.append(copy.deepcopy(y_test[i]))
            
    print("Total matches found : %d " % count)
    
    return xt,np.asarray(yt)

In [274]:
X_A, y_A = extractor(signatureA, X_test, y_test)
X_A.shape, len(y_A)

Total matches found : 5 


((5, 57), 5)

In [275]:
X_B, y_B = extractor(signatureB, X_test, y_test)
X_B.shape, len(y_B)

Total matches found : 13 


((13, 57), 13)

In [276]:
X_C, y_C = extractor(signatureC, X_test, y_test)
X_C.shape, len(y_C)

Total matches found : 23 


((23, 57), 23)

In [277]:
X_D, y_D = extractor(signatureD, X_test, y_test)
X_D.shape, len(y_D)

Total matches found : 24 


((24, 57), 24)

In [278]:
len(X_test)

65

In [279]:
# import all the filenames
from allfunctions import *

In [280]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:><:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>
<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:><:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>
<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:><:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>
<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:><:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>


In [281]:
# Read (frome file) the tuned model for each of 10 Model types
# Apply each of the 10 *best* models to 4 pairs(X_A, y_A) ...
# Generate 4 graphs with 10 test RMSEs on them (train RMSE, test RMSE, mean test set value)

In [282]:
from sklearn.metrics import mean_squared_error 
import numpy as np

def rmse_scorer(model, X, y): 
    y_predict = model.predict(X)
    k = np.sqrt(mean_squared_error(y, y_predict))
    return k

In [283]:
# Only for testing : reload the test dataset

with open('datasetIPPD.pickle', 'rb') as handle:
        data = pickle.load(handle)

X_test = data['X_test']
y_test = data['y_test']

# 1 LassoCVModel

In [284]:
modelFile = '4C4_lasso_tuned.pickle'
with open(modelFile, 'rb') as handle:
    model = pickle.load(handle)

In [285]:
#A
which = 'A'
X = X_A
y = y_A
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#B
which = 'B'
X = X_B
y = y_B
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#C
which = 'C'
X = X_C
y = y_C
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#D
which = 'D'
X = X_D
y = y_D
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#Full Test Set
which = 'Full Test Set'
X = X_test
y = y_test
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

A   || model : Lasso(alph
A   || test_rmse_reporting: 1500.708453
A   || test_mean_y_comparing: 117.836000

B   || model : Lasso(alph
B   || test_rmse_reporting: 871.644211
B   || test_mean_y_comparing: 2189.093846

C   || model : Lasso(alph
C   || test_rmse_reporting: 808.751399
C   || test_mean_y_comparing: 1703.226087

D   || model : Lasso(alph
D   || test_rmse_reporting: 770.530605
D   || test_mean_y_comparing: 1307.537500

Full Test Set   || model : Lasso(alph
Full Test Set   || test_rmse_reporting: 880.826425
Full Test Set   || test_mean_y_comparing: 1532.346154



# 2 OMPCVModel

In [286]:
modelFile = '4C4_OMPC_tuned.pickle'
with open(modelFile, 'rb') as handle:
    model = pickle.load(handle)

In [287]:
#A
which = 'A'
X = X_A
y = y_A
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#B
which = 'B'
X = X_B
y = y_B
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#C
which = 'C'
X = X_C
y = y_C
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#D
which = 'D'
X = X_D
y = y_D
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#Full Test Set
which = 'Full Test Set'
X = X_test
y = y_test
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

A   || model : Orthogonal
A   || test_rmse_reporting: 456.241910
A   || test_mean_y_comparing: 117.836000

B   || model : Orthogonal
B   || test_rmse_reporting: 689.197578
B   || test_mean_y_comparing: 2189.093846

C   || model : Orthogonal
C   || test_rmse_reporting: 109.644166
C   || test_mean_y_comparing: 1703.226087

D   || model : Orthogonal
D   || test_rmse_reporting: 170.507475
D   || test_mean_y_comparing: 1307.537500

Full Test Set   || model : Orthogonal
Full Test Set   || test_rmse_reporting: 354.963614
Full Test Set   || test_mean_y_comparing: 1532.346154



# 3 GradientBoostingCVModel

In [288]:
modelFile = '4C4_boosting_tuned.pickle'
with open(modelFile, 'rb') as handle:
    model = pickle.load(handle)

In [289]:
#A
which = 'A'
X = X_A
y = y_A
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#B
which = 'B'
X = X_B
y = y_B
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#C
which = 'C'
X = X_C
y = y_C
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#D
which = 'D'
X = X_D
y = y_D
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#Full Test Set
which = 'Full Test Set'
X = X_test
y = y_test
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

A   || model : GradientBo
A   || test_rmse_reporting: 99.034277
A   || test_mean_y_comparing: 117.836000

B   || model : GradientBo
B   || test_rmse_reporting: 2091.025469
B   || test_mean_y_comparing: 2189.093846

C   || model : GradientBo
C   || test_rmse_reporting: 1700.962307
C   || test_mean_y_comparing: 1703.226087

D   || model : GradientBo
D   || test_rmse_reporting: 1309.349458
D   || test_mean_y_comparing: 1307.537500

Full Test Set   || model : GradientBo
Full Test Set   || test_rmse_reporting: 1591.230073
Full Test Set   || test_mean_y_comparing: 1532.346154



# 4 Random Forest

In [290]:
modelFile = '4C4_rforest_tuned.pickle'
with open(modelFile, 'rb') as handle:
    model = pickle.load(handle)

In [291]:
#A
which = 'A'
X = X_A
y = y_A
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#B
which = 'B'
X = X_B
y = y_B
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#C
which = 'C'
X = X_C
y = y_C
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#D
which = 'D'
X = X_D
y = y_D
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#Full Test Set
which = 'Full Test Set'
X = X_test
y = y_test
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

A   || model : RandomFore
A   || test_rmse_reporting: 833.665902
A   || test_mean_y_comparing: 117.836000

B   || model : RandomFore
B   || test_rmse_reporting: 1402.511618
B   || test_mean_y_comparing: 2189.093846

C   || model : RandomFore
C   || test_rmse_reporting: 1101.283163
C   || test_mean_y_comparing: 1703.226087

D   || model : RandomFore
D   || test_rmse_reporting: 787.786025
D   || test_mean_y_comparing: 1307.537500

Full Test Set   || model : RandomFore
Full Test Set   || test_rmse_reporting: 1051.270504
Full Test Set   || test_mean_y_comparing: 1532.346154



# 5 RidgeCVModel

In [292]:
modelFile = '4C4_ridge_tuned.pickle'
with open(modelFile, 'rb') as handle:
    model = pickle.load(handle)

In [293]:
#A
which = 'A'
X = X_A
y = y_A
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#B
which = 'B'
X = X_B
y = y_B
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#C
which = 'C'
X = X_C
y = y_C
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#D
which = 'D'
X = X_D
y = y_D
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#Full Test Set
which = 'Full Test Set'
X = X_test
y = y_test
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

A   || model : Ridge(alph
A   || test_rmse_reporting: 213881193.697564
A   || test_mean_y_comparing: 117.836000

B   || model : Ridge(alph
B   || test_rmse_reporting: 245914900.514946
B   || test_mean_y_comparing: 2189.093846

C   || model : Ridge(alph
C   || test_rmse_reporting: 795990821.172131
C   || test_mean_y_comparing: 1703.226087

D   || model : Ridge(alph
D   || test_rmse_reporting: 179669658.880540
D   || test_mean_y_comparing: 1307.537500

Full Test Set   || model : Ridge(alph
Full Test Set   || test_rmse_reporting: 501727324.115041
Full Test Set   || test_mean_y_comparing: 1532.346154



# 6 ElasticNetCVModel

In [294]:
modelFile = '4C4_elasticNet_tuned.pickle'
with open(modelFile, 'rb') as handle:
    model = pickle.load(handle)

In [301]:
#A
which = 'A'
X = X_A
y = y_A
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#B
which = 'B'
X = X_B
y = y_B
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#C
which = 'C'
X = X_C
y = y_C
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#D
which = 'D'
X = X_D
y = y_D
######################################################################
reporting_testscore = rmse_scorer(model, X, y)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

#Full Test Set
which = 'Full Test Set'
X = X_test
y = y_test
######################################################################
reporting_testscore = rmse_scorer(model, X_test, y_test)

print('%s   || model : %s' % (which,str(model)[:10]))
print('%s   || test_rmse_reporting: %f' % (which,reporting_testscore))
print('%s   || test_mean_y_comparing: %f' % (which, y.mean()))
print()

A   || model : ElasticNet
A   || test_rmse_reporting: 6802326.658347
A   || test_mean_y_comparing: 117.836000

B   || model : ElasticNet
B   || test_rmse_reporting: 4626028.355249
B   || test_mean_y_comparing: 2189.093846

C   || model : ElasticNet
C   || test_rmse_reporting: 19646572.024725
C   || test_mean_y_comparing: 1703.226087

D   || model : ElasticNet
D   || test_rmse_reporting: 4765595.341818
D   || test_mean_y_comparing: 1307.537500

Full Test Set   || model : ElasticNet
Full Test Set   || test_rmse_reporting: 12361441.810858
Full Test Set   || test_mean_y_comparing: 1532.346154



In [297]:
######################################################

# 7 SVR - a - Poly Kernel

In [296]:
modelFile = '4C4_****_tuned.pickle'
with open(modelFile, 'rb') as handle:
    model = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: '4C4_****_tuned.pickle'

# 8 SVR - b - Sigmoid Kernel

In [ ]:
modelFile = '4C4_*****_tuned.pickle'
with open(modelFile, 'rb') as handle:
    model = pickle.load(handle)

# 9 SVR - c - Linear Kernel

In [ ]:
modelFile = '4C4_*****_tuned.pickle'
with open(modelFile, 'rb') as handle:
    model = pickle.load(handle)

# 10 SVR - d - rbf Kernel

In [ ]:
modelFile = '4C4_*****_tuned.pickle'
with open(modelFile, 'rb') as handle:
    model = pickle.load(handle)